# Word Embeddings
## Word embedding - origins and fundamentals
-    collective name for a set of language modeling and feature learning techings in natural language processing (NLP)
-    one-hot encoding is early example, but doesn't work
-    other examples borrow from Information Retrieval (IR): Term Frequency-Inverse Document Frequency (TF-IDF), Latent Semantic Analysis (LSA), and topic modeling

### Distributed representations
-    